In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import time
import gc
import tensorflow as tf
import transformers
import pandas as pd
import numpy as np
from keras.utils import np_utils
import json
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stanford-natural-language-inference-corpus/README.txt
/kaggle/input/stanford-natural-language-inference-corpus/snli_1.0_test.csv
/kaggle/input/stanford-natural-language-inference-corpus/snli_1.0_train.csv
/kaggle/input/stanford-natural-language-inference-corpus/snli_1.0_dev.csv


In [2]:
train=pd.read_csv("/kaggle/input/stanford-natural-language-inference-corpus/snli_1.0_train.csv")                  
test=pd.read_csv("/kaggle/input/stanford-natural-language-inference-corpus/snli_1.0_test.csv")
dev=pd.read_csv("/kaggle/input/stanford-natural-language-inference-corpus/snli_1.0_dev.csv")

In [3]:
train = train.dropna(subset = ['sentence2'])
train = train[train["gold_label"] != "-"]
test = test[test["gold_label"] != "-"]
dev = dev[dev["gold_label"] != "-"]
train.index = range(1,len(train)+1)
dev.index = range(1,len(dev)+1)
test.index = range(1,len(test)+1)
train['sentence1']

1         A person on a horse jumps over a broken down a...
2         A person on a horse jumps over a broken down a...
3         A person on a horse jumps over a broken down a...
4                     Children smiling and waving at camera
5                     Children smiling and waving at camera
                                ...                        
549357                  Four dirty and barefooted children.
549358                  Four dirty and barefooted children.
549359    A man is surfing in a bodysuit in beautiful bl...
549360    A man is surfing in a bodysuit in beautiful bl...
549361    A man is surfing in a bodysuit in beautiful bl...
Name: sentence1, Length: 549361, dtype: object

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
training_X = {'sentence1': train["sentence1"],'sentence2': train["sentence2"]}
dev_X = {'sentence1': dev["sentence1"],'sentence2': dev["sentence2"]}
testing_X={'sentence1': test["sentence1"],'sentence2': test["sentence2"]}

training_Y=[]
dev_Y=[]
test_Y=[]
for i in range(1,len(train['gold_label'])+1):
          if(train["gold_label"][i]=='entailment'):
              training_Y.append([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
          if(train["gold_label"][i]=='neutral'):
              training_Y.append([0,1,0,0,0,0,0,0,0,0,0,0,0,0,0])
          if(train["gold_label"][i]=='contradiction'):
              training_Y.append([0,0,1,0,0,0,0,0,0,0,0,0,0,0,0])

for i in range(1,len(dev['gold_label'])+1):
          if(dev["gold_label"][i]=='entailment'):
              dev_Y.append([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
          if(dev["gold_label"][i]=='neutral'):
              dev_Y.append([0,1,0,0,0,0,0,0,0,0,0,0,0,0,0])
          if(dev["gold_label"][i]=='contradiction'):
              dev_Y.append([0,0,1,0,0,0,0,0,0,0,0,0,0,0,0])       
              
for i in range(1,len(test['gold_label'])+1):
          if(test["gold_label"][i]=='entailment'):
              test_Y.append([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
          if(test["gold_label"][i]=='neutral'):
              test_Y.append([0,1,0,0,0,0,0,0,0,0,0,0,0,0,0])
          if(test["gold_label"][i]=='contradiction'):
              test_Y.append([0,0,1,0,0,0,0,0,0,0,0,0,0,0,0]) 

In [8]:
tokenizer = Tokenizer(num_words=1000, oov_token="<OOV>")
tokenizer.fit_on_texts(np.array(training_X['sentence1']))
word_index = tokenizer.word_index

training_sequences_X_1 = tokenizer.texts_to_sequences(np.array(training_X['sentence1'])) 

training_padded_X_1=(pad_sequences(training_sequences_X_1,maxlen=15,padding='post',truncating='post'))
training_sequences_X_2 = tokenizer.texts_to_sequences(np.array(training_X['sentence2'])) 


training_padded_X_2=(pad_sequences(training_sequences_X_2,maxlen=15,padding='post',truncating='post'))
training_padded_X=[]
for i in range(0,len(training_padded_X_1)):
  training_padded_X.append(np.absolute(training_padded_X_1[i]-training_padded_X_2[i]))

testing_sequences_X_1 = tokenizer.texts_to_sequences(testing_X['sentence1']) 
testing_padded_X_1=pad_sequences(testing_sequences_X_1,maxlen=15,padding='post',truncating='post')
testing_sequences_X_2 = tokenizer.texts_to_sequences(testing_X['sentence2']) 
testing_padded_X_2=pad_sequences(testing_sequences_X_2,maxlen=15,padding='post',truncating='post')
testing_padded_X=[]
for i in range(0,len(testing_sequences_X_1)):
  
      testing_padded_X.append(np.absolute(testing_padded_X_1[i]-testing_padded_X_2[i]))
dev_sequences_X_1 = tokenizer.texts_to_sequences(dev_X['sentence1']) 
dev_padded_X_1=pad_sequences(dev_sequences_X_1,maxlen=15,padding='post',truncating='post')
dev_sequences_X_2 = tokenizer.texts_to_sequences(dev_X['sentence2']) 
dev_padded_X_2=pad_sequences(dev_sequences_X_2,maxlen=15,padding='post',truncating='post')
dev_padded_X=[]
for i in range(0,len(dev_padded_X_1)):
  
      dev_padded_X.append(np.absolute(dev_padded_X_1[i]-dev_padded_X_2[i]))

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(1000, 16, input_length=15),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,input_shape=(549367,15),recurrent_dropout=0.2,return_sequences=True)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [14]:
training_padded_X = np.array(training_padded_X)
training_Y = np.array(training_Y)
testing_padded_X = np.array(testing_padded_X)
test_Y = np.array(test_Y)
history = model.fit(np.array(training_padded_X), np.array(training_Y), epochs=3, validation_data=(np.array(dev_padded_X), np.array(dev_Y)), verbose=1)
test_pred = model.predict(testing_padded_X, batch_size=128)
test_acc = (np.argmax(test_pred, axis=1) == np.argmax(test_Y, axis=1)).sum()/test_Y.shape[0] * 100

Epoch 1/3
17168/17168 [==============================] - 312s 18ms/step - loss: 0.1215 - accuracy: 0.9362 - val_loss: 0.1211 - val_accuracy: 0.9365
Epoch 2/3
17168/17168 [==============================] - 309s 18ms/step - loss: 0.1206 - accuracy: 0.9368 - val_loss: 0.1199 - val_accuracy: 0.9375
Epoch 3/3
17168/17168 [==============================] - 318s 19ms/step - loss: 0.1200 - accuracy: 0.9372 - val_loss: 0.1196 - val_accuracy: 0.9374


In [13]:
loss, accuracy = model.evaluate(testing_padded_X, test_Y, batch_size=64)

154/154 [==============================] - 1s 5ms/step - loss: 0.1214 - accuracy: 0.9363


In [18]:
import nltk
import re
import pprint
from nltk import Tree
from sklearn.model_selection import train_test_split
tagged_sentence = nltk.corpus.treebank.tagged_sents(tagset='universal')
print("Number of Tagged Sentences ",len(tagged_sentence))
tagged_words=[tup for sent in tagged_sentence for tup in sent]
vocab=set([word for word,tag in tagged_words])
tags=set([tag for word,tag in tagged_words])
N1_X=[]
N1_Y=[]

for i in range(0,3914):
    sent=[]
    taga=[]
    for j in tagged_sentence[i]:
        
        sent.append(j[0])
        
        taga.append(j[1])
    N1_X.append(sent)
    N1_Y.append(taga)
    

training_sequences_N1_X = tokenizer.texts_to_sequences(np.array(N1_X)) 
training_padded_N1_X=pad_sequences(training_sequences_N1_X,maxlen=15,padding='post',truncating='post')
training_padded_N1_X[20]
train_X=training_padded_N1_X[:1957]
validation_X=training_padded_N1_X[1957:]


Number of Tagged Sentences  3914


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [29]:
num_N1_Y=[]
for i in range(0,3914):
    num_reps=[]
    for j in N1_Y[i]:
        if(j=='.'):
            num_reps.append([1,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
        elif(j=='ADJ'):
            num_reps.append([0,1,0,0,0,0,0,0,0,0,0,0,0,0,0])
        elif(j=='ADP'):
            num_reps.append([0,0,1,0,0,0,0,0,0,0,0,0,0,0,0])
        elif(j=='ADV'):
            num_reps.append([0,0,0,1,0,0,0,0,0,0,0,0,0,0,0])
        elif(j=='CONJ'):
            num_reps.append([0,0,0,0,1,0,0,0,0,0,0,0,0,0,0])
        elif(j=='DET'):
            num_reps.append([0,0,0,0,0,1,0,0,0,0,0,0,0,0,0])
        elif(j=='NOUN'):
            num_reps.append([0,0,0,0,0,0,1,0,0,0,0,0,0,0,0])
        elif(j=='NUM'):
            num_reps.append([0,0,0,0,0,0,0,1,0,0,0,0,0,0,0])
        elif(j=='PRON'):
            num_reps.append([0,0,0,0,0,0,0,0,1,0,0,0,0,0,0])
        elif(j=='PRT'):
            num_reps.append([0,0,0,0,0,0,0,0,0,1,0,0,0,0,0])
        elif(j=='VERB'):
            num_reps.append([0,0,0,0,0,0,0,0,0,0,1,0,0,0,0])
        elif(j=='X'):
            num_reps.append([0,0,0,0,0,0,0,0,0,0,0,1,0,0,0])
        else:
            pass
    num_N1_Y.append(num_reps)

train_Y=num_N1_Y[:1957]
validation_Y=num_N1_Y[1957:]
    

In [30]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(1000, 16, input_length=15),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,input_shape=(549367,15),recurrent_dropout=0.2,return_sequences=True)),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

history_probe = model.fit(np.array(train_X), np.array(train_Y),validation_data=(np.array(validation_X),np.array(validation_Y)), epochs=1 , verbose=1)
test_pred_probe = model.predict(testing_X, batch_size=128)
test_pred_probe[1]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).